# Notebook to Template Model Merging with SLERP

In [3]:
!python3 -m pip install --upgrade pip
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.3 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Cloning into 'mergekit'...
remote: Enumerating objects: 1928, done.
remote: Counting objects: 100% (1191/1191), done.
remote: Compressing objects: 100% (511/511), done.
remote: Total 1928 (delta 940), reused 777 (delta 679), pack-reused 737
Receiving objects: 100% (1928/1928), 572.86 KiB | 5.67 MiB/s, done.
Resolving deltas: 100% (1318/1318), done.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.6 which is incompatible.
momepy 0.7.0

In [4]:
from huggingface_hub import notebook_login as hfl
hfl()

In [16]:
config = """
slices:
    - sources:
        - model: google/codegemma-7b-it
          layer_range: [0, 28]
        - model: codellama/CodeLlama-7b-Instruct-hf
          layer_range: [0, 28]
parameters:
    t:
        - filter: self_attn
          value: [0, 0.5, 0.3, 0.7, 1]
        - filter: mlp
          value: [1, 0.5, 0.7, 0.3, 0]
        - value: 0.5 # Fallback for the rest of the tensors
base_model: google/codegemma-7b-it
merge_method: slerp
dtype: float16
"""

In [17]:
with open("config.yaml", "w") as f:
    f.write(config)

In [18]:
!mergekit-yaml config.yaml output-dir --allow-crimes --copy-tokenizer --out-shard-size 1B --low-cpu-memory --write-model-card --lazy-unpickle

Executing graph:   1%|▏                        | 9/1272 [00:11<27:23,  1.30s/it]
Traceback (most recent call last):
  File "/opt/conda/bin/mergekit-yaml", line 8, in <module>
    sys.exit(main())
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/kaggle/working/mergekit/mergekit/options.py", line 82, in wrapper
    f(*args, **kwargs)
  File "/kaggle/working/mergekit/mergekit/scripts/run_yaml.py", line 47, in main
    run_merge(
  File "/kaggle/working/mergekit/mergekit/merge.py", line 92, in run_merge
    for _task, value in exec.run(quiet=opt